# Plot Project Car Data in Real Time

In [ ]:
import serial
import json
from types import SimpleNamespace 

import numpy as np
import matplotlib.pyplot as plt

import time

PORT = "/pathtoport"
BAUD = 9600

### Read Data

Use the pySerial library to read data from USB serial and save it as an array.
The data is in the form of a JSON object with the sensor outputs

In [ ]:
def getData():
    with serial.Serial(
        PORT,
        BAUD,
        timeout=1,
    ) as s:
        dataString = s.readline()
        data = json.loads(dataString, object_hook=lambda d: SimpleNamespace(**d))
        sensorData = np.array(data.sensor)
        return sensorData

### Plot Data

Use matplotlib matshow to plot the array as a heatmap

In [ ]:
from IPython import display

while(True):
    display.clear_output(wait=True)
    plt.figure()
    plt.title("IR Sensor Output")
    data = np.expand_dims(getData(), axis=0)
    color_map = plt.imshow(data)
    color_map.set_cmap("Blues_r")
    plt.colorbar()
    plt.show()
    time.sleep(BAUD**-1)